# CF - ALS (by Spark)

## Algorithm Theory

### Matrix factorization for collaborative filtering problem

Matrix factorization tries to find latent factors that represent intrinsic user and item attributes in a lower dimension. 

1. **Prediction Formula**

$$\hat r_{u,i} = q_{i}^{T}p_{u}$$

where:
- $\hat r_{u,i}$ is the predicted ratings for user $u$ and item $i$
- $q_{i}$ and $p_{u}$ are latent factors for item and user, respectively.

---> The goal is to find these latent factors $q_{i}^{T}$ and $p_{u}$ that best approximate the observed ratings.

2. **Optimization Problem**

The challenge to the matrix factorization problem is to find $q_{i}^{T}$ and $p_{u}$. This is achieved by methods such as **matrix decomposition**. A learning approach is therefore developed to converge the decomposition results close to the observed ratings as much as possible. Furthermore, to avoid overfitting issue, the learning process is **regularized**. 

***The Goal is minimize the prediction error***:

$$\min\sum(r_{u,i} - q_{i}^{T}p_{u})^2 + \lambda(||q_{i}||^2 + ||p_{u}||^2)$$

where $\lambda$ is a the regularization parameter that controls the trade-off between fitting the data well and keeping the model simple. 

**Trường hợp phải tính toán rating dựa trên interaction của KH với items - Implicit Feedback**

In case **explict ratings** are not available, **implicit ratings** which are usually derived from users' historical interactions with the items (e.g., *clicks*, *views*, *purchases*, etc.). To account for such implicit ratings, the original matrix factorization algorithm can be formulated as 

$$\min\sum c_{u,i}(p_{u,i} - q_{i}^{T}p_{u})^2 + \lambda(||q_{i}||^2 + ||p_{u}||^2)$$

where:
- $c_{u,i}=1+\alpha r_{u,i}$: là trọng số cho từng tương tác. Nó phụ thuộc vào giá trị $r_{u,i}$, là một biểu thị của sự yêu thích của người dùng $u$ cho item $i$, giúp mô hình nhận biết rằng không phải tất cả các tương tác đều có giá trị như nhau. Một tương tác với số lần nhấp nhiều hơn sẽ được coi là quan trọng hơn so với một tương tác có ít lần nhấp.
- $p_{u,i}=1$ if $r_{u,i}>0$ else $p_{u,i}=0$ (if $r_{u,i}=0$): Điều này có nghĩa là chỉ khi có tương tác, chúng ta mới tính đến phản hồi đó
  > $r_{u,i}$ is a numerical representation of users' preferences (e.g., number of clicks, etc.). 

### Alternating Least Square (ALS)

Owing to the term of $q_{i}^{T}p_{u}$ the loss function is non-convex. Gradient descent method can be applied but this will incur expensive computations. An Alternating Least Square (ALS) algorithm was therefore developed to overcome this issue. 

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTGkKzxWWEfGMc2qp2b4wQXdgkMNxt6ijs9tg&s">

The basic idea of ALS is to learn one of $q$ and $p$ at a time for optimization while keeping the other as constant. This makes the objective at each iteration convex and solvable. The alternating between $q$ and $p$ stops when there is convergence to the optimal. It is worth noting that this iterative computation can be parallelised and/or distributed, which makes the algorithm desirable for use cases where the dataset is large and thus the user-item rating matrix is super sparse (as is typical in recommendation scenarios). A comprehensive discussion of ALS and its distributed computation can be found [here](http://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf).

### Spark Mllib implementation

The matrix factorization algorithm is available as `ALS` module in [Spark `ml`](https://spark.apache.org/docs/latest/ml-collaborative-filtering.html) for DataFrame or [Spark `mllib`](https://spark.apache.org/docs/latest/mllib-collaborative-filtering.html) for RDD. 

* The uniqueness of ALS implementation is that it distributes the matrix factorization model training by using "Alternating Least Square" method. 
* In the training method, there are parameters that can be selected to control the model performance.
* Both explicit and implicit ratings are supported by Spark ALS model.

## Setup enviroment and parameters

In [1]:
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import sys

# from matplotlib import pyplot as plt
# import numpy as np
import pandas as pd
# import seaborn as sns

import pyspark

# from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS

# import pyspark.sql.functions as F
# from pyspark.sql.functions import col
# from pyspark.ml.tuning import CrossValidator
# from pyspark.sql.types import StructType, StructField
# from pyspark.sql.types import FloatType, IntegerType, LongType
from pyspark.sql.types import StructType, StructField, StringType, FloatType

# from recommenders.datasets import movielens
from recommenders.utils.spark_utils import start_or_get_spark
from recommenders.evaluation.spark_evaluation import (
    # SparkRankingEvaluation,
    SparkRatingEvaluation,
)
# from recommenders.tuning.parameter_sweep import generate_param_grid
# from recommenders.datasets.spark_splitters import spark_random_split

print(f"System version: {sys.version}")
print(f"Pandas version: {pd.__version__}")
print(f"PySpark version: {pyspark.__version__}")

System version: 3.11.5 (main, Sep 11 2023, 08:31:25) [Clang 14.0.6 ]
Pandas version: 2.2.2
PySpark version: 3.5.2


**model parameters**

In [2]:
# model hyper-parameters
# rank of the factorization
RANK = 10
# Số lượng vòng lặp tối đa
MAX_ITER = 15
# Hệ số kiểm soát regularization
REG_PARAM = 0.05

# Number of recommended items
K = 15

# rating columns
USER_COL = "acnt_no"
ITEM_COL = "symbol"
RATING_COL = "final_score"

# Path to the CSV file
train_datapath = r"data/label/data_process_UserItemRating_min5items_train.csv"
test_datapath = r"data/label/data_process_UserItemRating_min5items_train.csv"

**Spark session**

In [3]:
spark = start_or_get_spark("ALS", memory="16g")
# spark = SparkSession.builder.appName("ALS recommendation spark session").getOrCreate()
spark.conf.set("spark.sql.analyzer.failAmbiguousSelfJoin", "false")

24/08/20 19:40:03 WARN Utils: Your hostname, ats-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.1.3 instead (on interface en0)
24/08/20 19:40:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/20 19:40:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
# STOP SESSION
spark.stop()

## Train model

**Load data**

In [4]:
# Define the schema
schema = StructType(
    [
        StructField(USER_COL, StringType(), True),
        StructField(ITEM_COL, StringType(), True),
        StructField(RATING_COL, FloatType(), True),
    ]
)

# Read the CSV file into a DataFrame with the specified schema
train = spark.read.csv(train_datapath, sep="|", header=True, schema=schema)
test = spark.read.csv(test_datapath, sep="|", header=True, schema=schema)

# Show the DataFrame to verify
test.show(3)

+--------+------+-----------+
| acnt_no|symbol|final_score|
+--------+------+-----------+
|00018956|   BID|       0.02|
|00024968|   BID|       0.03|
|00025170|   VIX|      3.924|
+--------+------+-----------+
only showing top 3 rows



In [13]:
train.describe().show()

+-------+-----------------+------+-------------------+
|summary|          acnt_no|symbol|        final_score|
+-------+-----------------+------+-------------------+
|  count|           532012|532012|             532012|
|   mean|59160.27250137215|  NULL|0.22697969638126742|
| stddev|52375.70211050739|  NULL| 0.8522782496254795|
|    min|         00000002|   AAA|              0.002|
|    max|         00186799|   YTC|             12.869|
+-------+-----------------+------+-------------------+



**Preprocessing pipeline**

Do mô hình ALS spark yêu câu dữ liệu **user** và **item** ở dạng `numeric`, nên cần xây dựng pipeline chuyển đổi dữ liệu

In [5]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

# Chuyển đổi cột user
user_indexer = StringIndexer(inputCol=USER_COL, outputCol="userIndex")

# Chuyển đổi cột item
item_indexer = StringIndexer(inputCol=ITEM_COL, outputCol="itemIndex")

**Setup model**

It is worth noting that Spark ALS model allows dropping cold users to favor a robust evaluation with the testing data. In case there are cold users, Spark ALS implementation allows users to drop cold users in order to make sure evaluations on the prediction results are sound.

In [6]:
als = ALS(
    maxIter=MAX_ITER,
    rank=RANK,
    regParam=REG_PARAM,
    userCol="userIndex",
    itemCol="itemIndex",
    ratingCol=RATING_COL,
    nonnegative=True,
    coldStartStrategy="drop",
)

**Train model**

In [7]:
# make pipeline
model_pipeline = Pipeline(stages=[user_indexer, item_indexer, als])

# fit transform train data
model = model_pipeline.fit(train)

24/08/20 19:40:33 WARN DAGScheduler: Broadcasting large task binary with size 1106.3 KiB
24/08/20 19:40:33 WARN DAGScheduler: Broadcasting large task binary with size 1108.6 KiB
Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=
24/08/20 19:40:33 WARN DAGScheduler: Broadcasting large task binary with size 1110.1 KiB


CodeCache: size=131072Kb used=24376Kb max_used=24383Kb free=106696Kb
 bounds [0x00000001071e8000, 0x00000001089d8000, 0x000000010f1e8000]
 total_blobs=9483 nmethods=8496 adapters=898
 compilation: disabled (not enough contiguous free space left)


24/08/20 19:40:34 WARN DAGScheduler: Broadcasting large task binary with size 1111.4 KiB
24/08/20 19:40:34 WARN DAGScheduler: Broadcasting large task binary with size 1110.4 KiB
24/08/20 19:40:35 WARN DAGScheduler: Broadcasting large task binary with size 1111.7 KiB
24/08/20 19:40:35 WARN DAGScheduler: Broadcasting large task binary with size 1112.4 KiB
24/08/20 19:40:35 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/08/20 19:40:36 WARN DAGScheduler: Broadcasting large task binary with size 1115.6 KiB
24/08/20 19:40:36 WARN DAGScheduler: Broadcasting large task binary with size 1117.0 KiB
24/08/20 19:40:36 WARN DAGScheduler: Broadcasting large task binary with size 1118.4 KiB
24/08/20 19:40:36 WARN DAGScheduler: Broadcasting large task binary with size 1119.8 KiB
24/08/20 19:40:36 WARN DAGScheduler: Broadcasting large task binary with size 1121.1 KiB
24/08/20 19:40:36 WARN DAGScheduler: Broadcasting large task binary with size 1122.5 KiB
24/

In [10]:
train_trans = model.transform(train)

**Model Prediction**

In [8]:
predictions = model.transform(test)

## Model Evaluation

In [10]:
predictions.show(3)

24/08/20 19:43:05 WARN DAGScheduler: Broadcasting large task binary with size 1089.5 KiB
24/08/20 19:43:05 WARN DAGScheduler: Broadcasting large task binary with size 1164.4 KiB
24/08/20 19:43:05 WARN DAGScheduler: Broadcasting large task binary with size 1163.0 KiB


+--------+------+-----------+---------+---------+----------+
| acnt_no|symbol|final_score|userIndex|itemIndex|prediction|
+--------+------+-----------+---------+---------+----------+
|00102155|   VNM|      0.068|   6654.0|     14.0|0.04956831|
|00002006|   NTP|       0.03|   7754.0|    322.0|0.51148427|
|00032726|   MBB|      0.026|   1580.0|      0.0|0.17433679|
+--------+------+-----------+---------+---------+----------+
only showing top 3 rows



In [11]:
evaluations = SparkRatingEvaluation(
    predictions,
    predictions,
    col_user="userIndex",
    col_item="itemIndex",
    col_rating=RATING_COL,
    col_prediction="prediction",
)

print(
    "RMSE score = {}".format(evaluations.rmse()),
    "MAE score = {}".format(evaluations.mae()),
    "R2 score = {}".format(evaluations.rsquared()),
    "Explained variance score = {}".format(evaluations.exp_var()),
    sep="\n",
)

24/08/20 19:43:32 WARN DAGScheduler: Broadcasting large task binary with size 1088.5 KiB
24/08/20 19:43:32 WARN DAGScheduler: Broadcasting large task binary with size 1164.4 KiB
24/08/20 19:43:32 WARN DAGScheduler: Broadcasting large task binary with size 1163.0 KiB
24/08/20 19:43:34 WARN DAGScheduler: Broadcasting large task binary with size 1088.5 KiB
24/08/20 19:43:34 WARN DAGScheduler: Broadcasting large task binary with size 1164.4 KiB
24/08/20 19:43:34 WARN DAGScheduler: Broadcasting large task binary with size 1163.0 KiB
24/08/20 19:43:35 WARN DAGScheduler: Broadcasting large task binary with size 1089.0 KiB
24/08/20 19:43:35 WARN DAGScheduler: Broadcasting large task binary with size 1164.4 KiB
24/08/20 19:43:35 WARN DAGScheduler: Broadcasting large task binary with size 1163.0 KiB
24/08/20 19:43:35 WARN DAGScheduler: Broadcasting large task binary with size 1088.8 KiB
24/08/20 19:43:38 WARN DAGScheduler: Broadcasting large task binary with size 1236.0 KiB
24/08/20 19:43:40 WAR

RMSE score = 0.5246021666414594
MAE score = 0.14616747823206927
R2 score = 0.6211230665514835
Explained variance score = 0.6218804488068812


## Appendix

### Transform label

In [7]:
import pandas as pd

train = pd.read_csv(
    r"data/label/data_process_UserItemRating_min5items_train.csv",
    sep="|",
    dtype={"acnt_no": str},
)
test = pd.read_csv(
    r"data/label/data_process_UserItemRating_min5items_test.csv",
    sep="|",
    dtype={"acnt_no": str},
)